# Data Science Bootcamp Group 2 Project 1
# Medicare In-Patient Data Analysis: What is Ailing the Elderly?
### By: Brian Connel, Jeff Rosal, Ashok Bidani, and Michael Stewart

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
from over65 import over65

## We obtained our data from data.cms.gov (https://data.cms.gov/browse?category=Medicare%20-%20Inpatient)
### We chose this data source since they are pre-cleaned reports from government agencies (that likely use them internally). We specifically focused on in-patient discharge information due to the large amount of data in each table and that the data were available longitudinally (2011-2015)
### We speculated that we could look at three major types of data:
#### 1) What types of illnesses predominate among Medicare discharges?
#### 2) What is the scale and variability of Medicare payments by state?
#### 3) How have these changed over time?

In [3]:
# Declare filepaths to our CSV files
csv_path_2011 = "../data/Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv"
csv_path_2012 = "../data/Medicare_Provider_Charge_Inpatient_DRG100_FY2012.csv"
csv_path_2013 = "../data/Medicare_Provider_Charge_Inpatient_DRG100_FY2013.csv"
csv_path_2014 = "../data/Medicare_Provider_Charge_Inpatient_DRGALL_FY2014.csv"
csv_path_2015 = "../data/Medicare_Provider_Charge_Inpatient_DRGALL_FY2015.csv"
population_path = "../data/NST-EST2017-01.csv"

# Read our data into pandas
df_2011 = pd.read_csv(csv_path_2011)
df_2011['year'] = '2011'
df_2012 = pd.read_csv(csv_path_2012)
df_2012['year'] = '2012'
df_2013 = pd.read_csv(csv_path_2013)
df_2013['year'] = '2013'
df_2014 = pd.read_csv(csv_path_2014)
df_2014['year'] = '2014'
df_2015 = pd.read_csv(csv_path_2015)
df_2015['year'] = '2015'

df = pd.concat([df_2011,df_2012,df_2013,df_2014,df_2015])

## We realized early on that while there was an abundance of state-by-state data, none of it was normalized and that would impact statistical analysis.
### To remedy this, Brian found a CSV of populations by year that also included regional information. By loading this into a dataframe, we could pull the population values into our main dataframe and normalize discharges by population

In [4]:
# Read the population data into a pandas dataframe
df_pop = pd.read_csv(population_path)

## We also realized early on that the DRG, Medicare's 'grouped diagnosis' term, is both a blessing a curse.
### DRG's simplify noise in the data by grouping codes with similar diagnoses together. However, there exists a plethora of DRG's that are similar for our purposes, but distinct due to things like complications or comorbidities.
### To remedy this, Brian deployed RegEx to search through the dataframe of DRG's and replace the patterns with emptry strings:

In [5]:
# just to show original DRGs
drg_titles = set(df['DRG Definition'].unique())
# print(len(drg_titles),drg_titles)

In [6]:
#cleaning DRG titles to create 'mod_DRG Definition' series
patterns= [' W/O CC/MCC',' W/O MCC',' W MCC', ' W CC', ' MCC', '/MCC']
df['step1_mod_DRG Definition'] = df['DRG Definition'].replace(patterns, "", regex=True)
# knock off 'xxx - ', always 3 digits + ' - '
df['mod_DRG Definition'] = df['step1_mod_DRG Definition'].apply(lambda x : x[6:])
df.drop(columns=['step1_mod_DRG Definition'],inplace=True)

##### This for loop takes a minute, test code outside to minimize waiting or consider saving as csv

In [7]:
# Rename the USPS & Provider State columns to state for simplification of labels
df = df.rename(columns={'Provider State':'state'})
df_pop = df_pop.rename(columns={'USPS':'state'})
# Reindex df_pop by state for using loc to look up values later
df_pop = df_pop.set_index('state')
# Re-index to add blank column
list_of_columns = [column for column in df.columns.values]
list_of_columns.append('population')
#list_of_columns.append('division')
#list_of_columns.append('region')
list_of_columns.append('population')
df = df.reindex(columns=list_of_columns)
# Create an empty list that will be filled with the state/year population from df_pop
# the list will then be used to fill the empty pandas column 'population'
list_of_pops = []
list_of_divisions = []
list_of_regions = []
list_of_over65 = []
for index,row in df.iterrows():
    _state_year_population = df_pop.loc[row['state'],row['year']]
    #_state_division = df_pop.loc[row['state'],['Division']]
    #_state_region = df_pop.loc[row['state'],['Region']]
    _state_year_population = _state_year_population.replace(',','')
    list_of_pops.append(_state_year_population)
    #list_of_divisions.append(_state_division)
    #list_of_regions.append(_state_region)
    #print(f"Row#{index},{row['state']},{row['year']}:",_state_year_population)
df['population']=list_of_pops
#df['division']=list_of_divisions
#df['region']=list_of_regions
print("I'm done!")

I'm done!


##### End slow codeblock

In [10]:
# Save working df to csv option:
#df.to_csv('df_working.csv',sep=',')

df = pd.read_csv('df_working.csv')
df.head()

,Unnamed: 0,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,state,Provider Zip Code,Hospital Referral Region (HRR) Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,year,mod_DRG Definition,population,division,region,population.1
0,0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,32963.07692,5777.241758,4763.736264,2011,EXTRACRANIAL PROCEDURES,4798649,NaN,NaN,4798649
1,1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85714,5787.571429,4976.714286,2011,EXTRACRANIAL PROCEDURES,4798649,NaN,NaN,4798649
2,2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,37560.37500,5434.958333,4453.791667,2011,EXTRACRANIAL PROCEDURES,4798649,NaN,NaN,4798649
3,3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,13998.28000,5417.560000,4129.160000,2011,EXTRACRANIAL PROCEDURES,4798649,NaN,NaN,4798649
4,4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,31633.27778,5658.333333,4851.444444,2011,EXTRACRANIAL PROCEDURES,4798649,NaN,NaN,4798649


In [ ]:
# Create an empty column for "1e5s of pop" and "relative discharges"
list_of_columns = [column for column in df.columns.values]
list_of_columns.append('1e5s of pop')
list_of_columns.append('relative discharges')
df = df.reindex(columns=list_of_columns)
# Create a list of relative (100,000s) populations and add to dataframe
list_of_rel_pops = []
for pop in list_of_pops:
    pop = int(pop) / 100000
    list_of_rel_pops.append(pop)

df['1e5s of pop'] = list_of_rel_pops
# Do the same to create a list of relative discharges
list_of_rel_discharges =[]
for discharges, relpop in zip(df['Total Discharges'],df['1e5s of pop']):
    rel_disch = discharges / float(relpop)
    list_of_rel_discharges.append(rel_disch)
df['relative discharges'] = list_of_rel_discharges

In [ ]:
#### vvvvvv Can we remove this? vvvvv #####
drgs = df.groupby(['mod_DRG Definition','state'])
#drgs_per_state = drgs['state'].count()
#drgs_per_state_table.head()
# drgs_per_state

In [ ]:
# drgs_per_state_table
sns.set_style('white')
f,ax = plt.subplots(figsize=(10,5))
sns.heatmap(drgs_per_state_table, cmap='Blues');
#### ^^^^^ Can we remove this? ^^^^^ #####

## We wanted to begin by analyzing nationwide trends in discharges and payments to get an idea of any large-scale changes.

In [ ]:
# Groupby year in the df to sum
yeargroup = df.groupby('year')
revised_yeargroup = yeargroup.sum()
reduced_yeargroup = revised_yeargroup.loc[:, ["Total Discharges",'Average Medicare Payments']]
reduced_yeargroup.head()

In [ ]:
# Create axis object that contains the sum of all discharges each year over 2011-2015
sns.set_style('darkgrid')
fig, early_discharges = plt.subplots()
early_discharges = reduced_yeargroup['Total Discharges'].plot(kind='line',color='purple')
ax1 = early_discharges
ax1.set_xlabel("Year")
#Produce second axis object that shares the x axis with ax1
ax2 = ax1.twinx()
ax2 = reduced_yeargroup['Average Medicare Payments'].plot(kind='line',color='green')
#Title chart and axes
tot_disch = mpatches.Patch(color='purple', label='Total Discharges')
plt.legend(handles=[tot_disch,sum_pay])
sum_pay = mpatches.Patch(color='green', label='Sum of Medicare Payments($Bn')
ax1.set_ylabel('Total Discharges')
ax2.set_ylabel('Total Medicare Payments ($)')
ax1.set_title('Total Discharges and Medicare Payments 2011-2015')
print(ax1.get_xticks())
plt.show()

## It's notable there is a relatively large inflection beginning in 2013-2014.
### By reading the documentation, we found out that earlier datasets 2011-2013 contained only the top 100 DRG's (though ranking schema was not disclosed). After 2014, data would be available for all DRG's. This is why the number of discharges trend so closely.

# ***Note Brian Wanted Regional Charts Here
## When we plotted the total of discharges by state we initially saw several large outliers, which could be explained by their large state populations having a larger number of elderly people.
## We then chose to normalize by discharges per 100,000 people and see what variance remained

In [ ]:
stategroup = df.groupby('state')
revised_stategroup = stategroup.sum()
reduced_stategroup = revised_stategroup.loc[:, ["Total Discharges"]]

state_discharges = reduced_stategroup.plot(kind='bar')
state_discharges.set_xlabel("Provider State")
state_discharges.set_ylabel("Total Discharges (2011-2015)")
state_discharges.set_title('Medicare Discharges by State 2011-2015')

plt.show()

In [ ]:
stategroup = df.groupby('state')
revised_stategroup = stategroup.sum()
reduced_stategroup_rel = revised_stategroup.loc[:, ["relative discharges"]]

state_rel_discharges = reduced_stategroup_rel.plot(kind='bar',figsize=(15,10))
state_rel_discharges.set_xlabel("Provider State")
state_rel_discharges.set_ylabel("Discharges per 100,000 people")
state_rel_discharges.set_title('Discharges per 100,000 people by State 2011-2015')
plt.show()

In [ ]:
yearly_payments = revised_yeargroup.loc[:, ["Average Medicare Payments"]]
yearly_payments = yearly_payments.rename(columns = {"Average Medicare Payments": "Total Medicare Payment"})

totalmedicarebystate = df.groupby('state')['Average Medicare Payments'].sum()

ax1 = totalmedicarebystate.plot(kind='bar', title ='Total Medicare by State',figsize=(15,10),legend=False, fontsize=12)
ax1.set_xlabel('state',fontsize=12)
ax1.set_ylabel('medicare',fontsize=12)

In [ ]:
diagnosesgroup = df.groupby(['Provider State','mod_DRG Definition'])

newdiagnoses = diagnosesgroup['Total Discharges'].sum()
newdiagnoses.sort_values(inplace = True, ascending=[False]) 
countsdictionary = {}
drgdictionary = {}
for state,drg in newdiagnoses.index.values:
   drgdictionary[state] = drg
   countsdictionary[state] = newdiagnoses[state].iloc[0], newdiagnoses[state].index.values[0]
   
newdiagnoses3 = pd.DataFrame.from_dict(countsdictionary,orient='index')    
newdiagnoses4 = pd.DataFrame.from_dict(countsdictionary) 

# newdiagnoses3['Average Medicare Payments per State pop'] = 
# newdiagnoses3.index.values.transform(lambda x :LookUpPop(x) )

for state in newdiagnoses3.index.values:
#     print(LookUpPop(state).iloc[0],state)
    newdiagnoses3[:,'per'] = LookUpPop(state).iloc[0]
    
#newdiagnoses3.loc[state]/


newdiagnoses3.tail()

## At this point, we could begin comparing population-normalized discharge rates between states
### To accomplish this, we continued to use pandas groupby and loc tools to identify values for each state